# Проект: "Выбираем автомобиль правильно"

## Прогнозирование стоимости автомобиля.

### Изначальное кол-во колонок в датасете без преобразований:

1. bodyType - тип кузова
1. brand - марка автомобиля
1. car_url - ссылки
1. color - цвет автомобиля
1. complectation_dict - комплектация
1. description - описание
1. engineDisplacement - объем двигателя
1. enginePower - мощность двигателя
1. equipment_dict - комплектация(eng)
1. fuelType - тип топлива
1. image - фотография автомобиля
1. mileage - пробег
1. modelDate - дата модели
1. model_info - информация о модели
1. model_name - название модели
1. name - двигатель и тип трансмиссии, привод
1. numberOfDoors - количество дверей
1. parsing_unixtime - дата время парсинга
1. priceCurrency - валюта
1. productionDate - дата производства
1. sell_id - идентификатор продажи
1. super_gen - дополнительная информация
1. vehicleConfiguration - конфигурация автомобиля
1. vehicleTransmission - трансмиссия, коробка передач
1. vendor - производитель
1. Владельцы
1. Владение
1. ПТС
1. Привод
1. Руль
1. Состояние
1. Таможня

In [2]:
#!pip install -U dataprep

In [3]:
import numpy as np 
import pandas as pd
from pandas import Series
#import dataprep.eda

from tqdm.notebook import tqdm

from sklearn.model_selection import KFold
from catboost import CatBoostRegressor
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV

In [4]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [5]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [6]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

# Setup

In [93]:
VERSION    = 16
DIR_TRAIN  = '../input/dataframe-x4/' # подключил к ноутбуку внешний датасет
DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%

# Data

In [8]:
!ls '../input'

dataframe-x4			       sf-dst-car-price-prediction
parsing-all-moscow-auto-ru-09-09-2020


In [94]:
train = pd.read_csv(DIR_TRAIN+'data_from_open_sources.csv') # датасет для обучения модели
test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

In [10]:
train.head(5)

,bodyType,brand,color,description,engineDisplacement,enginePower,fuelType,mileage,modelDate,model_name,...,productionDate,sell_id,super_gen,vehicleTransmission,Владельцы,ПТС,Привод,Руль,Состояние,price
0,внедорожник,MERCEDES,белый,"Авто в идеальном состоянии, любые проверки. Ре...",5.0,296.0,бензин,191000.0,1990.0,G_KLASSE,...,2000.0,1.103330e+09,"{'id': '20495263', 'name': '500', 'nameplate':...",автоматическая,2.0,Оригинал,полный,левый,Не требует ремонта,1250000.0
1,внедорожник,INFINITI,чёрный,"2011год\nмаксимальная комплектация, активный к...",3.7,333.0,бензин,150000.0,2008.0,FX,...,2011.0,1.103594e+09,"{'id': '6237442', 'name': '37', 'nameplate': '...",автоматическая,3.0,Оригинал,полный,левый,Не требует ремонта,1220000.0
2,внедорожник,VOLVO,серый,"Пожалуйста, получите лучшее предложение у Ваше...",2.0,190.0,бензин,3372.0,2017.0,XC40,...,2021.0,1.103527e+09,"{'id': '21311114', 'displacement': 1969, 'engi...",автоматическая,1.0,Оригинал,полный,левый,Не требует ремонта,3350000.0
3,универсал,VOLVO,чёрный,- Автомобиль в отличном состоянии.\n- ТС пол...,2.0,163.0,дизель,171583.0,2007.0,XC70,...,2011.0,1.103467e+09,"{'id': '9344702', 'displacement': 1984, 'engin...",автоматическая,3.0,Оригинал,передний,левый,Не требует ремонта,1110000.0
4,хэтчбек,HONDA,серебристый,ТЕХНИЧЕСКАЯ ГАРАНТИЯ 1 ГОД - KARSO…...\nГАРАНТ...,1.3,98.0,гибрид,80000.0,2011.0,INSIGHT,...,2013.0,1.103255e+09,"{'id': '20392550', 'displacement': 1339, 'engi...",вариатор,1.0,Оригинал,передний,правый,Не требует ремонта,699000.0


In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292033 entries, 0 to 292032
Data columns (total 23 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   bodyType             292033 non-null  object 
 1   brand                292033 non-null  object 
 2   color                292033 non-null  object 
 3   description          285842 non-null  object 
 4   engineDisplacement   291691 non-null  float64
 5   enginePower          292033 non-null  float64
 6   fuelType             292033 non-null  object 
 7   mileage              292033 non-null  float64
 8   modelDate            40202 non-null   float64
 9   model_name           186605 non-null  object 
 10  name                 292033 non-null  object 
 11  numberOfDoors        292033 non-null  float64
 12  priceCurrency        40202 non-null   object 
 13  productionDate       292033 non-null  float64
 14  sell_id              40202 non-null   float64
 15  super_gen        

In [12]:
test.head(5)

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2 LTR,105 N12,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,...,LIFTBACK ROBOT 1.2,роботизированная,EUROPEAN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,чёрный,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6 LTR,110 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,LIFTBACK MECHANICAL 1.6,механическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
2,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/superb/11...,серый,"{""id"":""20026336"",""name"":""Ambition"",""available_...","Все автомобили, представленные в продаже, прох...",1.8 LTR,152 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,...,LIFTBACK ROBOT 1.8,роботизированная,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
3,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,коричневый,"{""id"":""20803582"",""name"":""Ambition"",""available_...",КОМПЛЕКТ ЗИМНЕЙ (ЛЕТНЕЙ) РЕЗИНЫ ПО СЕЗОНУ В ПО...,1.6 LTR,110 N12,"{""cruise-control"":true,""roller-blind-for-rear-...",бензин,...,LIFTBACK AUTOMATIC 1.6,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
4,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,белый,NaN,ЛОТ: 01220889\nАвтопрага Север\n\nВы можете по...,1.8 LTR,152 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,LIFTBACK AUTOMATIC 1.8,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен


In [13]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   bodyType              34686 non-null  object
 1   brand                 34686 non-null  object
 2   car_url               34686 non-null  object
 3   color                 34686 non-null  object
 4   complectation_dict    6418 non-null   object
 5   description           34686 non-null  object
 6   engineDisplacement    34686 non-null  object
 7   enginePower           34686 non-null  object
 8   equipment_dict        24690 non-null  object
 9   fuelType              34686 non-null  object
 10  image                 34686 non-null  object
 11  mileage               34686 non-null  int64 
 12  modelDate             34686 non-null  int64 
 13  model_info            34686 non-null  object
 14  model_name            34686 non-null  object
 15  name                  34686 non-null

In [14]:
train.dropna(subset=['price'], inplace=True)

In [15]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 257154 entries, 0 to 257361
Data columns (total 23 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   bodyType             257154 non-null  object 
 1   brand                257154 non-null  object 
 2   color                257154 non-null  object 
 3   description          250972 non-null  object 
 4   engineDisplacement   256867 non-null  float64
 5   enginePower          257154 non-null  float64
 6   fuelType             257154 non-null  object 
 7   mileage              257154 non-null  float64
 8   modelDate            40202 non-null   float64
 9   model_name           151726 non-null  object 
 10  name                 257154 non-null  object 
 11  numberOfDoors        257154 non-null  float64
 12  priceCurrency        40202 non-null   object 
 13  productionDate       257154 non-null  float64
 14  sell_id              40202 non-null   float64
 15  super_gen        

Ну вот, теперь наш трейн похудел на 35 000 строк)))

In [16]:
train.isna().sum()

bodyType                    0
brand                       0
color                       0
description              6182
engineDisplacement        287
enginePower                 0
fuelType                    0
mileage                     0
modelDate              216952
model_name             105428
name                        0
numberOfDoors               0
priceCurrency          216952
productionDate              0
sell_id                216952
super_gen              216952
vehicleTransmission         0
Владельцы                8290
ПТС                       259
Привод                      0
Руль                        0
Состояние              216952
price                       0
dtype: int64

In [17]:
test.isna().sum()

bodyType                    0
brand                       0
car_url                     0
color                       0
complectation_dict      28268
description                 0
engineDisplacement          0
enginePower                 0
equipment_dict           9996
fuelType                    0
image                       0
mileage                     0
modelDate                   0
model_info                  0
model_name                  0
name                        0
numberOfDoors               0
parsing_unixtime            0
priceCurrency               0
productionDate              0
sell_id                     0
super_gen                   0
vehicleConfiguration        0
vehicleTransmission         0
vendor                      0
Владельцы                   0
Владение                22691
ПТС                         1
Привод                      0
Руль                        0
Состояние                   0
Таможня                     0
dtype: int64

In [18]:
test.bodyType = test.drop(test[test.bodyType == 'внедорожник открытый'].index)
test.bodyType = test.drop(test[test.bodyType == 'внедорожник 3 дв.'].index)
test['bodyType'] = test['bodyType'].fillna('внедорожник 5 дв.')

test.bodyType = test.drop(test[test.bodyType == 'хэтчбек 3 дв.'].index)
test['bodyType'] = test['bodyType'].fillna('хэтчбек 5 дв.')

test.bodyType = test.drop(test[test.bodyType == 'купе-хардтоп'].index)
test['bodyType'] = test['bodyType'].fillna('купе')

test.bodyType = test.drop(test[test.bodyType == 'седан-хардтоп'].index)
test.bodyType = test.drop(test[test.bodyType == 'седан 2 дв.'].index)
test['bodyType'] = test['bodyType'].fillna('седан')

test.bodyType = test.drop(test[test.bodyType == 'пикап полуторная кабина'].index)
test.bodyType = test.drop(test[test.bodyType == 'пикап одинарная кабина'].index)
test['bodyType'] = test['bodyType'].fillna('пикап двойная кабина')

#test.bodyType = test.drop(test[test.bodyType == 'тарга'].index)
#test['bodyType'] = test['bodyType'].fillna('родстер')
#Тáрга — тип автомобильного кузова легкового автомобиля, разновидность спортивного 2-местного родстера 
#с жёстко закреплённым лобовым стеклом, дугой безопасности сзади сидений, съёмной крышей и задним стеклом.
# Можно было бы поменять на значение "родстер", но оставлю как есть.

In [19]:
test['bodyType'] = [str(x).lower().replace('.', '') for x in test['bodyType']]
test['bodyType'] = [str(x).lower().replace('дв', '') for x in test['bodyType']]
test['bodyType'] = [str(x).lower().replace('5', '') for x in test['bodyType']]
test['bodyType'] = [str(x).lower().replace('кабина', '') for x in test['bodyType']]
test['bodyType'] = [str(x).lower().replace('ойная', '') for x in test['bodyType']]
test['bodyType'] = [str(x).lower().replace(' ', '') for x in test['bodyType']]

In [20]:
test['engineDisplacement'] = test['engineDisplacement'].apply(lambda x: '2.0 LTR' if x == ' LTR' else x)
test['engineDisplacement'] = test['engineDisplacement'].apply(lambda x: str(x).replace('LTR', ''))

Произведём такую же замену как и в трейне.

In [21]:
test['enginePower'] = test['enginePower'].apply(lambda x: x.replace(' N12', ''))

In [22]:
test['Владельцы'] = [str(x).lower().replace('или', '') for x in test['Владельцы']]
test['Владельцы'] = [str(x).lower().replace('более', '') for x in test['Владельцы']]
test['Владельцы'] = [str(x).lower().replace('владелец', '') for x in test['Владельцы']]
test['Владельцы'] = [str(x).lower().replace('владельца', '') for x in test['Владельцы']]

In [23]:
test['Руль'] = test['Руль'].apply(lambda x: x.lower())

In [24]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   bodyType              34686 non-null  object
 1   brand                 34686 non-null  object
 2   car_url               34686 non-null  object
 3   color                 34686 non-null  object
 4   complectation_dict    6418 non-null   object
 5   description           34686 non-null  object
 6   engineDisplacement    34686 non-null  object
 7   enginePower           34686 non-null  object
 8   equipment_dict        24690 non-null  object
 9   fuelType              34686 non-null  object
 10  image                 34686 non-null  object
 11  mileage               34686 non-null  int64 
 12  modelDate             34686 non-null  int64 
 13  model_info            34686 non-null  object
 14  model_name            34686 non-null  object
 15  name                  34686 non-null

In [25]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 257154 entries, 0 to 257361
Data columns (total 23 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   bodyType             257154 non-null  object 
 1   brand                257154 non-null  object 
 2   color                257154 non-null  object 
 3   description          250972 non-null  object 
 4   engineDisplacement   256867 non-null  float64
 5   enginePower          257154 non-null  float64
 6   fuelType             257154 non-null  object 
 7   mileage              257154 non-null  float64
 8   modelDate            40202 non-null   float64
 9   model_name           151726 non-null  object 
 10  name                 257154 non-null  object 
 11  numberOfDoors        257154 non-null  float64
 12  priceCurrency        40202 non-null   object 
 13  productionDate       257154 non-null  float64
 14  sell_id              40202 non-null   float64
 15  super_gen        

In [26]:
# Данные колонки не нужны (они не несут полезной и значимой информации, 
# либо дублируют её, либо много пропусков)
train_drop = ['name', 'priceCurrency', 'sell_id', 'super_gen','Состояние', 'modelDate', 'Владельцы']
train.drop(columns=train_drop, inplace=True)

test_drop = ['car_url', 'complectation_dict', 'equipment_dict', 'image','model_info'
            ,'name', 'parsing_unixtime', 'priceCurrency', 'super_gen'
            ,'vehicleConfiguration', 'Владение', 'Владельцы', 'Состояние', 'Таможня', 'modelDate']
test.drop(columns=test_drop, inplace=True)

In [27]:
# для корректной обработки признаков объединяем трейн и тест в один датасет
train['sample'] = 1 # помечаем где у нас трейн
test['sample'] = 0 # помечаем где у нас тест
test['price'] = 0 # в тесте у нас нет значения price, мы его должны предсказать, по этому пока просто заполняем нулями

train_data = train.append(test, sort=False).reset_index(drop=True) # объединяем

In [28]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291840 entries, 0 to 291839
Data columns (total 19 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   bodyType             291840 non-null  object 
 1   brand                291840 non-null  object 
 2   color                291840 non-null  object 
 3   description          285658 non-null  object 
 4   engineDisplacement   291553 non-null  object 
 5   enginePower          291840 non-null  object 
 6   fuelType             291840 non-null  object 
 7   mileage              291840 non-null  float64
 8   model_name           186412 non-null  object 
 9   numberOfDoors        291840 non-null  float64
 10  productionDate       291840 non-null  float64
 11  vehicleTransmission  291840 non-null  object 
 12  ПТС                  291580 non-null  object 
 13  Привод               291840 non-null  object 
 14  Руль                 291840 non-null  object 
 15  price            

In [29]:
train_data.isna().sum()

bodyType                    0
brand                       0
color                       0
description              6182
engineDisplacement        287
enginePower                 0
fuelType                    0
mileage                     0
model_name             105428
numberOfDoors               0
productionDate              0
vehicleTransmission         0
ПТС                       260
Привод                      0
Руль                        0
price                       0
sample                      0
sell_id                257154
vendor                 257154
dtype: int64

Не так много колонок с пропусками - это хорошо!

In [30]:
train_data.drop_duplicates()

,bodyType,brand,color,description,engineDisplacement,enginePower,fuelType,mileage,model_name,numberOfDoors,productionDate,vehicleTransmission,ПТС,Привод,Руль,price,sample,sell_id,vendor
0,внедорожник,MERCEDES,белый,"Авто в идеальном состоянии, любые проверки. Ре...",5,296,бензин,191000.0,G_KLASSE,5.0,2000.0,автоматическая,Оригинал,полный,левый,1250000.0,1,NaN,NaN
1,внедорожник,INFINITI,чёрный,"2011год\nмаксимальная комплектация, активный к...",3.7,333,бензин,150000.0,FX,5.0,2011.0,автоматическая,Оригинал,полный,левый,1220000.0,1,NaN,NaN
2,внедорожник,VOLVO,серый,"Пожалуйста, получите лучшее предложение у Ваше...",2,190,бензин,3372.0,XC40,5.0,2021.0,автоматическая,Оригинал,полный,левый,3350000.0,1,NaN,NaN
3,универсал,VOLVO,чёрный,- Автомобиль в отличном состоянии.\n- ТС пол...,2,163,дизель,171583.0,XC70,5.0,2011.0,автоматическая,Оригинал,передний,левый,1110000.0,1,NaN,NaN
4,хэтчбек,HONDA,серебристый,ТЕХНИЧЕСКАЯ ГАРАНТИЯ 1 ГОД - KARSO…...\nГАРАНТ...,1.3,98,гибрид,80000.0,INSIGHT,5.0,2013.0,вариатор,Оригинал,передний,правый,699000.0,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291835,седан,BMW,коричневый,Продается BMW 316i в отличном состоянии. Прода...,1.6,136,бензин,115000.0,3ER,4.0,2014.0,автоматическая,Оригинал,задний,левый,0.0,0,1.101369e+09,EUROPEAN
291836,седан,BMW,чёрный,Продаётся отличный автомобиль.,2.0,190,дизель,98000.0,5ER,4.0,2018.0,автоматическая,Оригинал,полный,левый,0.0,0,1.101370e+09,EUROPEAN
291837,седан,BMW,серый,Хорошее состояние,2.5,170,бензин,360000.0,5ER,4.0,1997.0,автоматическая,Дубликат,задний,левый,0.0,0,1.101365e+09,EUROPEAN
291838,внедорожник,BMW,коричневый,Автомобиль в идеальном состоянии . Куплен новы...,2.0,184,дизель,90500.0,X1,5.0,2013.0,автоматическая,Оригинал,полный,левый,0.0,0,1.101363e+09,EUROPEAN


## EDA

In [31]:
#dataprep.eda.plot(train_data, 'bodyType')

Внедорожники и седаны в превалирующем большинстве.

In [32]:
#dataprep.eda.plot(train_data, 'bodyType', 'price')

In [33]:
#dataprep.eda.plot(train_data, 'brand')

In [34]:
#dataprep.eda.plot(train_data, 'brand', 'price')

Сразу видно премиум сегмент)))

In [35]:
#dataprep.eda.plot(train_data, 'color')

Чёрных машин больше всех. Кому-то летом будет жарко)))

In [36]:
#dataprep.eda.plot(train_data, 'color', 'price')

In [37]:
#dataprep.eda.plot(train_data, 'description')

In [38]:
# сделаем признак отсутствующих описаний
#train_data['description_NaN'] = train_data['description'].apply(lambda x: 1 if x == 0 else 0)

In [39]:
#train_data['description'] = train_data['description'].fillna('unknown')

Заполним отсутствующие значения на "неизвестно"

##### Теперь можно подготовить колонку к составлению новых признаков

In [40]:
#train_data['description'] = train_data['description'].apply(lambda x: [str(i).lower() for i in x.split()])

In [41]:
#train_data['airbag'] = train_data.description.apply(lambda x: 
#                                                         1 if ('подушка' and 'безопасность') in x else 0)
#train_data['alloy_wheels'] = train_data.description.apply(lambda x: 
#                                                               1 if ('легкосплавный' and 'диск') in x else 0)
#train_data['heated_mirrors'] = train_data.description.apply(lambda x: 
#                                                                 1 if ('обогрев' and 'зеркало') in x else 0)
#train_data['central_locking'] = train_data.description.apply(lambda x: 
#                                                                  1 if ('центральный' and 'замок') in x else 0)
#train_data['on-board_computer'] = train_data.description.apply(lambda x: 
#                                                                    1 if ('бортовой' and 'компьютер') in x else 0)
#train_data['abs'] = train_data.description.apply(lambda x: 
#                                                      1 if ('антиблокировочный' and 'система') in x else 0)
#train_data['light_sensor'] = train_data.description.apply(lambda x: 
#                                                               1 if ('датчик' and 'свет') in x else 0)
#train_data['upholstery'] = train_data.description.apply(lambda x: 
#                                                             1 if ('обивка' and 'салон') in x else 0)
#train_data['heated_seat'] = train_data.description.apply(lambda x: 
#                                                              1 if ('подогрев' and 'сидение') in x else 0)
#train_data['power_steering'] = train_data.description.apply(lambda x: 
#                                                                 1 if ('усилитель' and 'руль') in x else 0)
#train_data['сruise_control'] = train_data.description.apply(lambda x: 
#                                                                 1 if ('круиз' and 'контроль') in x else 0)
#train_data['climate_control'] = train_data.description.apply(lambda x: 
#                                                                  1 if ('климат' and 'контроль') in x else 0)
#train_data['led_lights'] = train_data.description.apply(lambda x: 
#                                                             1 if ('светодиодный' and 'фара') in x else 0)
#train_data['fog_lights'] = train_data.description.apply(lambda x: 
#                                                             1 if ('противотуманный' and 'фара') in x else 0)
#train_data['leather']= train_data.description.apply(lambda x: 
#                                                         1 if ('темный' and 'салон') in x else 0)
#train_data['carter']= train_data.description.apply(lambda x: 
#                                                        1 if ('защита' and 'картера') in x else 0)
#train_data['door_closers']= train_data.description.apply(lambda x: 
#                                                              1 if ('доводчики' and 'дверей') in x else 0)
#train_data['rear_view_camera']= train_data.description.apply(lambda x: 
#                                                                  1 if ('камера' or 'видеокамера' and 'заднего' and 'вида') in x else 0)

In [42]:
#description_cols = ['rear_view_camera', 'door_closers', 'carter', 'leather', 'fog_lights',
#                    'led_lights', 'climate_control', 'сruise_control','power_steering',
#                    'heated_seat', 'upholstery', 'light_sensor', 'abs',
#                    'on-board_computer','central_locking', 'heated_mirrors', 'alloy_wheels', 'airbag']

In [43]:
#dataprep.eda.plot(train_data, 'engineDisplacement')

Больше всего авто с объёмом двигателя 2.0 и 1.6

In [44]:
#dataprep.eda.plot(train_data, 'engineDisplacement', 'price')

Авто с двигателем 2.9 дороже.

In [45]:
#train_data.engineDisplacement=train_data.engineDisplacement.fillna(2.0)
#train_data.engineDisplacement=train_data.engineDisplacement.astype('float64')

In [46]:
#dataprep.eda.plot(train_data, 'enginePower')

Машины с 249 л.с. и 150 л.с. в равной степени в большинстве.

In [47]:
#dataprep.eda.plot(train_data, 'enginePower', 'price')

Авто с 190 л.с. стоят дороже.

In [48]:
#train_data.enginePower=train_data.enginePower.astype('float64')

In [49]:
#dataprep.eda.plot(train_data, 'fuelType')

Бензиновых авто естественно больше, это очевидно и без графиков))) Посмотрим как изменится машиностроение в будущем) Возможно, относительно скоро, наши датасеты с данными будут уже не актуальны)

In [50]:
#dataprep.eda.plot(train_data, 'fuelType', 'price')

Авто на газе ещё не в моде, посмотрим, что будет через 10-15 лет.

In [51]:
#dataprep.eda.plot(train_data, 'mileage')

Распределение линейное, за исключением выбросов в начале и в конце. Всего скорее это связано с тем, что в первом случае машины новые, а во втором - старые, раритетные. 

In [52]:
#f_j = train[train.mileage == 0]
#f_j.productionDate.value_counts()

Делаем простой и очевидный вывод: Машины без пробега, потому что они новые стоят в салоне!

In [53]:
#dataprep.eda.plot(train_data, 'model_name')

Чем заполнить пустые значения? Попробую заменить на "неизвестно". Посмотрю в дальнейшем,важен ли признак для модели или его можно удалить.

In [54]:
#train_data['model_name'] = train_data['model_name'].fillna('unknown')
#train_data['model_name'] = train_data['model_name'].apply(lambda x: (x.split(' ')[0]).lower())

In [55]:
#dataprep.eda.plot(train_data, 'numberOfDoors')

Преобладают 4-х и 5-ти дверные авто.

In [56]:
#f_j = train_data[train_data.numberOfDoors == 0]
#f_j.model_name.value_counts()

Погуглив, оказалось, что у этой модели нет дверей. Так что ничего трогать не будем.

In [57]:
#dataprep.eda.plot(train_data, 'productionDate')

In [58]:
#dataprep.eda.plot(train_data, 'productionDate', 'price')

У машин после 1969-1971 г.г. идут выбросы. Будем считать, что авто до этих годов выпуска раритетные.

In [59]:
#dataprep.eda.plot(train_data, 'vehicleTransmission')

Больше всего авто на автоматической коробке передач.

In [60]:
#dataprep.eda.plot(train_data, 'vehicleTransmission', 'price')

In [61]:
#dataprep.eda.plot(train_data, 'ПТС')

In [62]:
#train_data.ПТС=train_data.ПТС.fillna('Оригинал')

In [63]:
#dataprep.eda.plot(train_data, 'Привод')

Дрифт уже не моде...                 Эх...)))

In [64]:
#dataprep.eda.plot(train_data, 'Руль')

Леворуких машин больше, европейский стандарт выигрывает)

In [65]:
#dataprep.eda.plot(train_data, 'vendor')

In [66]:
#train_data.loc[(train_data['brand'] == "MERCEDES") & (train_data.vendor.isnull()), 'vendor'] = 'EUROPEAN'
#train_data.loc[(train_data['brand'] == "BMW") & (train_data.vendor.isnull()), 'vendor'] = 'EUROPEAN'
#train_data.loc[(train_data['brand'] == "VOLKSWAGEN") & (train_data.vendor.isnull()), 'vendor'] = 'EUROPEAN'
#train_data.loc[(train_data['brand'] == "AUDI") & (train_data.vendor.isnull()), 'vendor'] = 'EUROPEAN'
#train_data.loc[(train_data['brand'] == "SKODA") & (train_data.vendor.isnull()), 'vendor'] = 'EUROPEAN'
#train_data.loc[(train_data['brand'] == "VOLVO") & (train_data.vendor.isnull()), 'vendor'] = 'EUROPEAN'
#train_data.loc[(train_data['brand'] == "NISSAN") & (train_data.vendor.isnull()), 'vendor'] = 'JAPANESE'
#train_data.loc[(train_data['brand'] == "TOYOTA") & (train_data.vendor.isnull()), 'vendor'] = 'JAPANESE'
#train_data.loc[(train_data['brand'] == "MITSUBISHI") & (train_data.vendor.isnull()), 'vendor'] = 'JAPANESE'
#train_data.loc[(train_data['brand'] == "HONDA") & (train_data.vendor.isnull()), 'vendor'] = 'JAPANESE'
#train_data.loc[(train_data['brand'] == "LEXUS") & (train_data.vendor.isnull()), 'vendor'] = 'JAPANESE'
#train_data.loc[(train_data['brand'] == "INFINITI") & (train_data.vendor.isnull()), 'vendor'] = 'JAPANESE'

In [67]:
#dataprep.eda.plot(train, 'price')

In [68]:
#train_data.info()

#### Создадим несколько новых признаков

In [69]:
#train_data['years_old'] = 2021 - train_data['productionDate']

In [70]:
#train_data['car_rarity']=train_data.years_old >= 50
#train_data['car_new']=train_data.years_old <= 3

In [71]:
#train_data['car_econom']=train_data.engineDisplacement <= 1.5
#train_data['car_premium']=train_data.engineDisplacement >= 3.0

In [72]:
#train_data['disp_power'] = train_data.engineDisplacement * train_data.enginePower
#train_data['mil_prod'] = train_data.mileage * train_data.productionDate
#train_data['disp_prod'] = train_data.engineDisplacement * train_data.productionDate
#train_data['power_prod'] = train_data.enginePower * train_data.productionDate

#new_columns = ['disp_power','mil_prod', 'disp_prod', 'power_prod']

In [73]:
#объединим все полученные признаки по категориям
#bin_cols = ['ПТС','Руль','car_rarity','car_new','car_econom','car_premium'] 
#cat_cols = ['brand', 'color', 'bodyType', 'enginePower', 'fuelType', 
#            'model_name', 'numberOfDoors', 'vehicleTransmission', 'engineDisplacement',
#            'Привод', 'vendor'] 
#num_cols = ['productionDate', 'years_old', 'mileage']

In [74]:
# Теперь прологарифмируем признаки.
#train_data['productionDate'] = np.log(train_data['productionDate'] + 1)
#train_data['years_old'] = np.log(train_data['years_old'] + 1)
#train_data['mileage'] = np.log(train_data['mileage'] + 1)
#train_data['price'] = np.log(train_data['price'] + 1)

In [75]:
#for colum in cat_cols:
#    train_data[colum] = train_data[colum].astype('category').cat.codes

In [76]:
# Нормализуем данные.
#scaler = StandardScaler()
#for col in num_cols:
#    train_data[col] = scaler.fit_transform(train_data[[col]])
#    
#for col in new_columns:
#    train_data[col] = scaler.fit_transform(train_data[[col]])    

In [77]:
# Закодируем бинарные переменные.
#label_encoder = LabelEncoder()

#for column in bin_cols:
#    train_data[column] = label_encoder.fit_transform(train_data[column])

#for column in description_cols:
#    train_data[column] = label_encoder.fit_transform(train_data[column])    

In [78]:
#imp_cat = Series(mutual_info_classif(train_data[bin_cols + cat_cols], train_data['price'], discrete_features =True), index = bin_cols + cat_cols) 
#imp_cat.sort_values(inplace = True) 
#imp_cat.plot(kind = 'barh', title = 'Значимость бинарных и категориальных признаков')

In [79]:
#imp_num = Series(f_classif(train_data[train_data.price.isna() == False][new_columns], train_data[train_data.price.isna() == False]['price'])[0], index = new_columns)
#imp_num.sort_values(inplace = True)
#imp_num.plot(kind = 'barh')

In [80]:
#imp_cat = Series(mutual_info_classif(train_data[description_cols], train_data['price'], discrete_features =True), index = description_cols) 
#imp_cat.sort_values(inplace = True) 
#imp_cat.plot(kind = 'barh')

In [81]:
#imp_cat = Series(mutual_info_classif(train_data[num_cols], train_data['price'], discrete_features =True), index = num_cols) 
#imp_cat.sort_values(inplace = True) 
#imp_cat.plot(kind = 'barh', title = 'Значимость числовых признаков')

### Data Preprocessing
##### Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [82]:
def preproc_data(df_input):
    
    
    df_output = df_input.copy()
    
    
    #
    def mape(y_true, y_pred):
        return np.mean(np.abs((y_pred-y_true)/y_true))

    
        
    # 
    df_output['description_NaN'] = df_output['description'].apply(lambda x: 1 if x == 0 else 0)
    df_output['description'] = df_output['description'].fillna('unknown')
    df_output['description'] = df_output['description'].apply(lambda x: [str(i).lower() for i in x.split()])
    
    df_output['airbag'] = df_output.description.apply(lambda x: 
                                                         1 if ('подушка' and 'безопасность') in x else 0)
    df_output['alloy_wheels'] = df_output.description.apply(lambda x: 
                                                                   1 if ('легкосплавный' and 'диск') in x else 0)
    df_output['heated_mirrors'] = df_output.description.apply(lambda x: 
                                                                     1 if ('обогрев' and 'зеркало') in x else 0)
    df_output['central_locking'] = df_output.description.apply(lambda x: 
                                                                      1 if ('центральный' and 'замок') in x else 0)
    df_output['on-board_computer'] = df_output.description.apply(lambda x: 
                                                                        1 if ('бортовой' and 'компьютер') in x else 0)
    df_output['abs'] = df_output.description.apply(lambda x: 
                                                          1 if ('антиблокировочный' and 'система') in x else 0)
    df_output['light_sensor'] = df_output.description.apply(lambda x: 
                                                                   1 if ('датчик' and 'свет') in x else 0)
    df_output['upholstery'] = df_output.description.apply(lambda x: 
                                                                 1 if ('обивка' and 'салон') in x else 0)
    df_output['heated_seat'] = df_output.description.apply(lambda x: 
                                                                  1 if ('подогрев' and 'сидение') in x else 0) 
    df_output['power_steering'] = df_output.description.apply(lambda x: 
                                                                     1 if ('усилитель' and 'руль') in x else 0)
    df_output['сruise_control'] = df_output.description.apply(lambda x: 
                                                                     1 if ('круиз' and 'контроль') in x else 0)
    df_output['climate_control'] = df_output.description.apply(lambda x: 
                                                                      1 if ('климат' and 'контроль') in x else 0)
    df_output['led_lights'] = df_output.description.apply(lambda x: 
                                                                 1 if ('светодиодный' and 'фара') in x else 0)
    df_output['fog_lights'] = df_output.description.apply(lambda x: 
                                                                 1 if ('противотуманный' and 'фара') in x else 0)
    df_output['leather']= df_output.description.apply(lambda x: 
                                                             1 if ('темный' and 'салон') in x else 0)
    df_output['carter']= df_output.description.apply(lambda x: 
                                                            1 if ('защита' and 'картера') in x else 0)
    df_output['door_closers']= df_output.description.apply(lambda x: 
                                                                  1 if ('доводчики' and 'дверей') in x else 0)
    df_output['rear_view_camera']= df_output.description.apply(lambda x: 
                                                                      1 if ('камера' or 'видеокамера' and 'заднего' and 'вида') in x else 0)
    
    description_cols = ['rear_view_camera', 'door_closers', 'carter', 'leather', 'fog_lights',
                    'led_lights', 'climate_control', 'сruise_control','power_steering',
                    'heated_seat', 'upholstery', 'light_sensor', 'abs',
                    'on-board_computer','central_locking', 'heated_mirrors', 'alloy_wheels', 'airbag']
    
    df_output.drop(['description'], axis=1, inplace=True)
    
    #
    df_output.engineDisplacement=df_output.engineDisplacement.fillna(2.0)
    df_output.engineDisplacement=df_output.engineDisplacement.astype('float64')
    
    #
    df_output.enginePower=df_output.enginePower.astype('float64')
    
    #
    df_output['model_name'] = df_output['model_name'].fillna('unknown')
    df_output['model_name'] = df_output['model_name'].apply(lambda x: (x.split(' ')[0]).lower())
    
    #
    df_output.ПТС=df_output.ПТС.fillna('Оригинал')
    
    #
    df_output.loc[(df_output['brand'] == "MERCEDES") & (df_output.vendor.isnull()), 'vendor'] = 'EUROPEAN'
    df_output.loc[(df_output['brand'] == "BMW") & (df_output.vendor.isnull()), 'vendor'] = 'EUROPEAN'
    df_output.loc[(df_output['brand'] == "VOLKSWAGEN") & (df_output.vendor.isnull()), 'vendor'] = 'EUROPEAN'
    df_output.loc[(df_output['brand'] == "AUDI") & (df_output.vendor.isnull()), 'vendor'] = 'EUROPEAN'
    df_output.loc[(df_output['brand'] == "SKODA") & (df_output.vendor.isnull()), 'vendor'] = 'EUROPEAN'
    df_output.loc[(df_output['brand'] == "VOLVO") & (df_output.vendor.isnull()), 'vendor'] = 'EUROPEAN'
    df_output.loc[(df_output['brand'] == "NISSAN") & (df_output.vendor.isnull()), 'vendor'] = 'JAPANESE'
    df_output.loc[(df_output['brand'] == "TOYOTA") & (df_output.vendor.isnull()), 'vendor'] = 'JAPANESE'
    df_output.loc[(df_output['brand'] == "MITSUBISHI") & (df_output.vendor.isnull()), 'vendor'] = 'JAPANESE'
    df_output.loc[(df_output['brand'] == "HONDA") & (df_output.vendor.isnull()), 'vendor'] = 'JAPANESE'
    df_output.loc[(df_output['brand'] == "LEXUS") & (df_output.vendor.isnull()), 'vendor'] = 'JAPANESE'
    df_output.loc[(df_output['brand'] == "INFINITI") & (df_output.vendor.isnull()), 'vendor'] = 'JAPANESE'
    
    #
    df_output['years_old'] = 2021 - df_output['productionDate']
    df_output['car_rarity']=df_output.years_old >= 50
    df_output['car_new']=df_output.years_old <= 3
    df_output['car_econom']=df_output.engineDisplacement <= 1.5
    df_output['car_premium']=df_output.engineDisplacement >= 3.0
    df_output['disp_power'] = df_output.engineDisplacement * df_output.enginePower
    df_output['mil_prod'] = df_output.mileage * df_output.productionDate
    df_output['disp_prod'] = df_output.engineDisplacement * df_output.productionDate
    df_output['power_prod'] = df_output.enginePower * df_output.productionDate

    new_columns = ['disp_power','mil_prod', 'disp_prod', 'power_prod']
    
    #
    bin_cols = ['ПТС','Руль','car_new','car_econom','car_premium','car_rarity'] 
    cat_cols = ['brand', 'color', 'bodyType', 'enginePower', 'fuelType', 
                'model_name', 'numberOfDoors', 'vehicleTransmission', 'engineDisplacement',
                'Привод', 'vendor'] 
    num_cols = ['productionDate', 'years_old', 'mileage']
    
    #
    df_output['productionDate'] = np.log(df_output['productionDate'] + 1)
    df_output['years_old'] = np.log(df_output['years_old'] + 1)
    df_output['mileage'] = np.log(df_output['mileage'] + 1)
    df_output['price'] = np.log(df_output['price'] + 1)
    
    
    #
    for colum in cat_cols:
        df_output[colum] = df_output[colum].astype('category').cat.codes
    
    scaler = StandardScaler()
    for col in num_cols:
        df_output[col] = scaler.fit_transform(df_output[[col]])

    for col in new_columns:
        df_output[col] = scaler.fit_transform(df_output[[col]])
        
    label_encoder = LabelEncoder()

    for column in bin_cols:
        df_output[column] = label_encoder.fit_transform(df_output[column])

    for column in description_cols:
        df_output[column] = label_encoder.fit_transform(df_output[column])
    
    
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis = 1, inplace=True)
    
    return df_output    

In [83]:
df_preproc = preproc_data(train_data)
df_preproc.sample(10)

,bodyType,brand,color,engineDisplacement,enginePower,fuelType,mileage,model_name,numberOfDoors,productionDate,...,rear_view_camera,years_old,car_rarity,car_new,car_econom,car_premium,disp_power,mil_prod,disp_prod,power_prod
151678,10,1,1,23,198,3,-3.136542,32,3,1.308294,...,1,-2.020905,0,1,0,1,0.433562,-1.378318,0.655873,0.635724
100437,0,7,12,18,126,0,0.257975,568,4,0.191378,...,1,0.137501,0,0,0,0,-0.221138,-0.372621,0.110218,-0.263015
136424,5,8,1,11,136,0,0.089778,568,4,0.471022,...,0,-0.161755,0,0,0,0,-0.419868,-0.809830,-0.633991,-0.138964
63759,10,5,1,28,159,0,0.561441,568,3,-4.605569,...,0,2.124470,0,0,0,1,0.374044,1.393925,1.113057,0.098031
277871,10,9,7,9,82,0,0.274171,147,3,0.051451,...,0,0.265321,0,0,0,0,-0.673274,-0.316258,-0.849922,-0.763060
223253,0,1,2,23,182,0,0.463432,620,4,-0.789571,...,0,0.849122,0,0,0,1,0.324614,0.635903,0.632033,0.416074
161932,0,6,11,13,103,0,-3.136542,380,4,1.168921,...,1,-1.477138,0,1,0,0,-0.484430,-1.378318,-0.415305,-0.512607
250303,0,7,1,9,146,0,0.212089,289,4,0.331235,...,1,-0.003797,0,0,0,0,-0.460219,-0.517369,-0.848226,-0.028932
172210,15,9,2,9,70,0,0.395546,147,4,-0.649227,...,0,0.767819,0,0,0,0,-0.718468,0.221821,-0.854160,-0.921373
55424,0,3,1,28,207,0,0.220808,568,4,0.889969,...,0,-0.792074,0,0,0,1,0.776548,-0.489721,1.185369,0.776222


In [84]:
df_preproc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291840 entries, 0 to 291839
Data columns (total 46 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   bodyType             291840 non-null  int8   
 1   brand                291840 non-null  int8   
 2   color                291840 non-null  int8   
 3   engineDisplacement   291840 non-null  int8   
 4   enginePower          291840 non-null  int16  
 5   fuelType             291840 non-null  int8   
 6   mileage              291840 non-null  float64
 7   model_name           291840 non-null  int16  
 8   numberOfDoors        291840 non-null  int8   
 9   productionDate       291840 non-null  float64
 10  vehicleTransmission  291840 non-null  int8   
 11  ПТС                  291840 non-null  int64  
 12  Привод               291840 non-null  int8   
 13  Руль                 291840 non-null  int64  
 14  price                291840 non-null  float64
 15  sample           

In [85]:
X = df_preproc.query('sample == 1').drop(['sample', 'price'], axis=1)
y = df_preproc.query('sample == 1')['price'] 
X_sub = df_preproc.query('sample == 0').drop(['sample', 'price'], axis=1)

In [86]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

In [87]:
cb = CatBoostRegressor(iterations = 3000,
                       random_seed = RANDOM_SEED,
                       eval_metric='MAPE',
                       silent=True,
                       learning_rate=0.15,depth=10,
                       l2_leaf_reg=5)
cb.fit(X_train, np.log(y_train),

     #cat_features=cat_features_ids,
     eval_set=(X_test, np.log(y_test)),
     verbose_eval=0,
     use_best_model=True
     )

#model.save_model('catboost.model')

predict_test = np.exp(cb.predict(X_test)) 
predict_submission = np.exp(cb.predict(X_sub))

print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Точность модели по метрике MAPE: 0.87%


In [88]:
#gb = GradientBoostingRegressor(min_samples_split=4, learning_rate=0.1, max_depth=10, n_estimators=5000)
#gb.fit(X_train, np.log(y_train))
#predict_test = np.exp(gb.predict(X_test))
#predict_submission = np.exp(gb.predict(X_sub))
#print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Хотел попробовать GradientBoostingRegressor, но по времени не успею... да и подобрать гиперпараметры надо ещё...

# Submission

In [102]:
sample_submission['price'] = predict_submission
sample_submission.to_csv('submission_1.csv', index=False)
sample_submission.head(10)

,sell_id,price
0,1100575026,1.336937
1,1100549428,1.381436
2,1100658222,1.370446
3,1100937408,1.351210
4,1101037972,1.352859
5,1100912634,1.360990
6,1101228730,1.334918
7,1100165896,1.302841
8,1100768262,1.456301
9,1101218501,1.362694



### Что еще можно сделать, чтоб улучшить результат:

* Спарсить свежие данные и данные с других сайтов (например дром.ру и авито)
* Посмотреть, что можно извлечь из признаков или как еще можно обработать признаки
* Сгенерировать новые признаки
* Попробовать другие алгоритмы и библиотеки ML
* Сделать Ансамбль моделей, Blending, Stacking

### На ЛБ показал 99. Я не могу понять почему? К большому сожалению, из-за форс-мажора, уже нет времени спросить и исправить. Очень жду помощи в обратной связи по проекту.